In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from dash import Dash, html, dcc, Input, Output

In [2]:
# Load the data
df = pd.read_csv('Peepal_Baba.csv')
df.head(5)

,Unnamed: 0,Title,Published_date,Views,Likes,Comments,Month
0,0,A must visit place in meerut.,2024-06-28,22337,2526,32,Jun
1,1,Everyone should grow their own vegetables.,2024-06-27,54986,4629,111,Jun
2,2,Environmental Activist!,2024-06-26,27057,2464,43,Jun
3,3,Sometimes get out from the crowd .,2024-06-25,29751,3107,49,Jun
4,4,Summer are here now!,2024-06-24,84573,5609,89,Jun


In [3]:
# Convert Published_date to datetime
df['Published_date'] = pd.to_datetime(df['Published_date'])
df['Year'] = df['Published_date'].dt.year

In [4]:
# Create the Dash app
app = Dash(__name__)

In [5]:
# Define the app layout
app.layout = html.Div(children=[
    html.H1(children="Peepal Baba Blog Analytics Dashboard"),

    # Dropdown for selecting the year
    dcc.Dropdown(
        id='year-dropdown',
        options=[{'label': year, 'value': year} for year in df['Year'].unique()],
        value=df['Year'].max(),
        clearable=False
    ),

    # Tabs for different chart types
    dcc.Tabs(id='chart-tabs', value='bar-chart', children=[
        dcc.Tab(label='Bar Chart', value='bar-chart'),
        dcc.Tab(label='Scatter Plot', value='scatter-plot'),
        dcc.Tab(label='Histogram', value='histogram')
    ]),

    # Div to display the selected chart
    html.Div(id='chart-display')
])

In [6]:
# Define the callback function for updating the chart
@app.callback(
    Output('chart-display', 'children'),
    Input('year-dropdown', 'value'),
    Input('chart-tabs', 'value')
)
def update_chart(selected_year, selected_chart):
    filtered_df = df[df['Year'] == selected_year]

    # Bar chart
    if selected_chart == 'bar-chart':
        fig = go.Figure()
        fig.add_trace(go.Bar(x=filtered_df['Month'], y=filtered_df['Views'], name='Views', marker_color='royalblue'))
        fig.add_trace(go.Bar(x=filtered_df['Month'], y=filtered_df['Likes'], name='Likes', marker_color='green'))
        fig.add_trace(go.Bar(x=filtered_df['Month'], y=filtered_df['Comments'], name='Comments', marker_color='orange'))
        fig.update_layout(
            title=f"Views, Likes, and Comments by Month ({selected_year})",
            xaxis_title="Month",
            yaxis_title="Count",
            barmode='group',
            plot_bgcolor='lightgrey',
            paper_bgcolor='whitesmoke'
        )
        return dcc.Graph(figure=fig)

    # Scatter plot
    elif selected_chart == 'scatter-plot':
        fig = px.scatter(filtered_df, x='Likes', y='Comments', color='Month', title=f"Likes vs Comments by Month ({selected_year})")
        fig.update_layout(
            plot_bgcolor='lightgrey',
            paper_bgcolor='whitesmoke'
        )
        return dcc.Graph(figure=fig)

    # Histogram
    elif selected_chart == 'histogram':
        fig = go.Figure()
        fig.add_trace(go.Histogram(x=filtered_df['Likes'], name='Likes', marker_color='blue'))
        fig.add_trace(go.Histogram(x=filtered_df['Comments'], name='Comments', marker_color='red'))
        fig.update_layout(
            title=f"Distribution of Likes and Comments ({selected_year})",
            xaxis_title="Count",
            yaxis_title="Frequency",
            bargap=0.2,
            plot_bgcolor='lightgrey',
            paper_bgcolor='whitesmoke'
        )
        return dcc.Graph(figure=fig)


In [7]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)